# Prepare the data 

In [1]:
import sys,os
import numpy as np
import random
import pandas as pd

In [4]:
file = 'D:\Evans microbial community\data_for_pipeline\metadata_full_MMPRNT_G5.LC_LUX_trunc_rar_016017_v2.txt'
richness = 'D:\Evans microbial community\data_for_pipeline\Rarefied_diversity_MMPRNT_G5_LC_LUX_016017.txt'
richness_type = 'full_Richness'
site = 'LC'
site_not_used = 'LUX'
cv_num = 10
test_size = 0.1 ### what the proportion of your data you want to hold out as test set, 
                ### which will never be seen when you train the model
save_model_name = 'Richness_model_build_on_LC.sav'
save_result_name = 'Result_of_Richness_model_build_on_LC.txt'
### add option to do regression or multi-class classification task
task = 'regression' # or 'multi-class'

In [ ]:
df = pd.read_csv(file, sep='\t', index_col = 0, header = 0)
rich = pd.read_csv(richness, sep='\t', index_col = 0, header = 0)

In [ ]:
df2 = df.drop(["collectionDate","siteID","UTM_Lat_Cord","UTM_Lon_Cord"],axis=1)

In [ ]:
### convert FeatStatus to features
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
FertStatus_1hot = cat_encoder.fit_transform(df2[["FertStatus"]])
df2.FertStatus = FertStatus_1hot.toarray()

In [ ]:
ML_matrix = pd.concat([rich[richness_type],df2], axis=1, sort = False)
ML_matrix = ML_matrix[ML_matrix[richness_type].notna()]

In [ ]:
np.any(np.isnan(ML_matrix[richness_type]))  ### If False, indicates no NaN in the value to be predicted

# Split data to training and test set

### 1. Split the data to target site and the other site

In [ ]:
df_3 = df.loc[ML_matrix.index.tolist()]
sampleID_target_site = df_3[df_3.siteID == site].index
sampleID_test_site = df_3[df_3.siteID != site].index
ML_matrix_target_site = ML_matrix.loc[sampleID_target_site,:]
ML_matrix_test_site = ML_matrix.loc[sampleID_test_site,:]

### 2. Split the data in target site to training and test set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(ML_matrix_target_site, \
                                       test_size=test_size, random_state=42)
X_train = train_set.drop(richness_type, axis=1) 
X_test = test_set.drop(richness_type, axis=1)
X_on_test_site = ML_matrix_test_site.drop(richness_type, axis=1)

y_train = train_set[richness_type]
y_test = test_set[richness_type]
y_on_test_site = ML_matrix_test_site[richness_type]

# Impute missing data using KNN, five Ks

### 1. drop features with >50% missing data

In [ ]:
Miss_count = X_train.count(0)
Col_to_drop = Miss_count[Miss_count <= 0.5*X_train.shape[0]].index.tolist()
Col_to_drop
X_train.drop(Col_to_drop,axis=1,inplace=True)
X_test.drop(Col_to_drop,axis=1,inplace=True)
X_on_test_site.drop(Col_to_drop,axis=1,inplace=True)

### 2. impute

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
class KNNImputer_Ks(BaseEstimator, TransformerMixin):
    def __init__(self, *Ks):
        self.Ks = Ks
    def fit(self, X,Ks):
        D_imputer = {}        
        for k in [3,4,5,6,7]:
            imputer = KNNImputer(n_neighbors=k)
            D_imputer[k] = imputer.fit(X)              
        return D_imputer
    def transform(self, X):
        Impute_train = {}
        for k in [3,4,5,6,7]:
            Impute_train[k] = pd.DataFrame(D_imputer[k].transform(X))
            Impute_train[k].index = X.index
            Impute_train[k].columns = X.columns 
            if k == 3:
                Imputed = Impute_train[k].copy(deep=True)
                Imputed.loc[:,:] = 0
            Imputed = Imputed.add(Impute_train[k],fill_value=0)
        return Imputed/5

In [ ]:
imputer_knn = KNNImputer_Ks()
D_imputer = imputer_knn.fit(X_train, Ks="3,4,5,6,7")
X_train_KNN = imputer_knn.transform(X_train)
X_test_KNN = imputer_knn.transform(X_test)
X_on_test_site_KNN =  imputer_knn.transform(X_on_test_site)

# Machine learning

### 1. GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
param_grid = {'max_depth':[3, 5, 10], \
              'max_features': [0.1, 0.5, 'sqrt', 'log2', None], \
              'n_estimators': [10, 100,500,1000]}
Reg_Mol = RandomForestRegressor()
grid_search = GridSearchCV(Reg_Mol, param_grid, cv=cv_num, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train_KNN, y_train)

### 2. cross-validation and prediction

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
parameter2use = grid_search.best_params_
Reg = RandomForestRegressor(n_estimators=parameter2use['n_estimators'],\
                            max_depth=parameter2use['max_depth'],\
                            max_features= parameter2use['max_features'],\
                            criterion='mse', random_state=42)

In [ ]:
# mse: Mean squared error regression loss
# evs: Explained variance regression score
# r2: (coefficient of determination) regression score. 
    # Best possible score is 1.0 and it can be negative 
    # (because the model can be arbitrarily worse). 
    # A constant model that always predicts the expected value of y, 
    # disregarding the input features, would get a R^2 score of 0.0.
# cor: Pearson Correlation Coefficient between true y and predicted y
cv_pred = cross_val_predict(estimator=Reg, X=X_train_KNN, y=y_train, cv=cv_num)
cv_mse = mean_squared_error(y_train, cv_pred)
cv_evs = explained_variance_score(y_train, cv_pred)
cv_r2 = r2_score(y_train, cv_pred)
cv_cor = np.corrcoef(np.array(y_train), cv_pred)[0,1]

### 3.Predict

In [ ]:
Reg.fit(X_train_KNN,y_train)
pred_train = Reg.predict(X_train_KNN)
train_mse = mean_squared_error(y_train, pred_train)
train_evs = explained_variance_score(y_train, pred_train)
train_r2 = r2_score(y_train, pred_train)
train_cor = np.corrcoef(np.array(y_train), pred_train)[0,1]

In [ ]:
pred_test = Reg.predict(X_test_KNN)
test_mse = mean_squared_error(y_test, pred_test)
test_evs = explained_variance_score(y_test, pred_test)
test_r2 = r2_score(y_test, pred_test)
test_cor = np.corrcoef(np.array(y_test), pred_test)[0,1]

In [ ]:
pred_on_test_site = Reg.predict(X_on_test_site_KNN)
test_site_mse = mean_squared_error(y_on_test_site, pred_on_test_site)
test_site_evs = explained_variance_score(y_on_test_site, pred_on_test_site)
test_site_r2 = r2_score(y_on_test_site, pred_on_test_site)
test_site_cor = np.corrcoef(np.array(y_on_test_site), pred_on_test_site)[0,1]

# Parse the results

### Save the fitted model

In [ ]:
import pickle
pickle.dump(Reg, open(save_model_name, 'wb'))

### write the hyperparameters

In [ ]:
out = open(save_result_name,'w')
out.write('The model is built using data from %s, and applied to %s and %s.\n\n'%(site,site,site_not_used))
out.write('There are %s training instances.\n'%X_train_KNN.shape[0])
out.write('There are %s test instances.\n'%X_test_KNN.shape[0])
out.write('There are %s instances in the other site.\n\n'%X_on_test_site_KNN.shape[0])
out.write('The model is built using RandomForestRegressor, with:\n')
out.write('\tn_estimators: %s\n'%parameter2use['n_estimators'])
out.write('\tmax_depth: %s\n'%parameter2use['max_depth'])
out.write('\tmax_features: %s\n\n'%parameter2use['max_features'])
out.write('There are %s feature used\n\n'%X_train_KNN.shape[1])

### Performance of models

In [ ]:
out.write('Prediction\tmse\tevs\tr2\tPCC\n')
out.write('CV\t%s\t%s\t%s\t%s\n'%(cv_mse,cv_evs,cv_r2,cv_cor))
out.write('Train\t%s\t%s\t%s\t%s\n'%(train_mse,train_evs,train_r2,train_cor))
out.write('Test\t%s\t%s\t%s\t%s\n'%(test_mse,test_evs,test_r2,test_cor))
out.write('Other_site\t%s\t%s\t%s\t%s\n\n'%(test_site_mse,test_site_evs,test_site_r2,test_site_cor))
out.close()

### Important features

In [ ]:
imp = pd.DataFrame({'Feature':X_train_KNN.columns, 'Importance':Reg.feature_importances_})
imp_sorted = imp.sort_values(by='Importance', ascending=False)
imp_sorted.to_csv(save_result_name.split('.txt')[0] + '_imp.txt', index=False, header=True,sep="\t")

In [ ]:
# Add the importance direction in the output